### Simple mujoco control
- UR5e
- Mujoco Python viewer
- Control with actuator name

In [1]:
import mujoco
import mujoco_viewer # new viewer
import numpy as np
import time

In [2]:
model_path = "../ur5e_mjcf/scene.xml"

In [3]:
# declare model & data
model = mujoco.MjModel.from_xml_path(model_path)
data = mujoco.MjData(model)

In [4]:
# mujoco model & data arrays

# number of control
number_of_control = model.nu
number_of_control


6

In [5]:
# Get name of controls
print(mujoco.mjtObj.mjOBJ_ACTUATOR) # enum constant to specify actuator object type
# Cpp enum: set of integer constants
# mjOBJ_ACTUATOR: actually integer constant to configure that given object is actuator
control_names = [mujoco.mj_id2name(model,mujoco.mjtObj.mjOBJ_ACTUATOR,ctrl_idx) for ctrl_idx in range(number_of_control)]
print(control_names)

mjtObj.mjOBJ_ACTUATOR
['shoulder_pan', 'shoulder_lift', 'elbow', 'wrist_1', 'wrist_2', 'wrist_3']


In [6]:
# get controller range
control_range = model.actuator_ctrlrange
print(control_range)

[[-6.2831  6.2831]
 [-6.2831  6.2831]
 [-3.1415  3.1415]
 [-6.2831  6.2831]
 [-6.2831  6.2831]
 [-6.2831  6.2831]]


In [7]:
"""
function: name and value array to input control  

- Use actuator name to control -> should get index & translate
"""

def apply_control(model, data, name, value):

    if len(name) != len(value):
        raise ValueError("length of name and value is different")

    # get control names from model
    control_names = [mujoco.mj_id2name(model,mujoco.mjtObj.mjOBJ_ACTUATOR,ctrl_idx) for ctrl_idx in range(model.nu)]

    # initialize control array
    ctrl_ = np.zeros(model.nu) # number of control

    # compare name and control_names to return index
    # idxs = []
    for i, n in enumerate(name):
        if n in control_names:
            idx = control_names.index(n)
            ctrl_[idx] = value[i]
            # idxs.append(idx)
        else:
            print(f"Name {n} is not included in actuator names, passing..")

    # assign value to corresponding control indices
    # for i, idx in enumerate(idxs):
    #     ctrl_[idx] = value[i]


    data.ctrl = ctrl_
    return None

apply_control(model, data, name=["shoulder_lift", "elbow"], value=[0.8, 0.8])
data.ctrl

array([0. , 0.8, 0.8, 0. , 0. , 0. ])

In [8]:
""" MAIN LOOP """

# create python viewer object
viewer = mujoco_viewer.MujocoViewer(model, data)

# data reset
mujoco.mj_resetData(model, data)

while True:
    if viewer.is_alive:

        # apply control
        apply_control(model, data, name=["shoulder_lift", "elbow"], value=[0.8, 0.8]) # now data 
        # control_signal = [0.] * 6
        
        mujoco.mj_step(model, data)
        viewer.render()

    else:
        break

# close
viewer.close()

Pressed ESC
Quitting.
